# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:

# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

from collections.abc import Iterable


# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#Load csv file created in Part 1
city_data_csv = "output_data/cities_myhw.csv"
#Turn csv into DataFrame
city_data_df = pd.read_csv(city_data_csv)
city_data_df


,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,porto novo,BJ,1643912886,6.4965,2.6036,84.02,64,75,8.39
1,kapaa,US,1643912886,22.0752,-159.3190,71.92,77,20,11.50
2,itarema,BR,1643912887,-2.9248,-39.9167,85.23,65,78,16.22
3,pyapon,MM,1643912888,16.2833,95.6833,68.25,65,14,5.97
4,rikitea,PF,1643912683,-23.1203,-134.9692,77.36,77,100,14.41
...,...,...,...,...,...,...,...,...,...
554,aden,YE,1643913160,12.7794,45.0367,78.13,60,0,9.44
555,rutland,US,1643913008,43.6167,-73.0162,42.24,94,100,1.01
556,revelstoke,CA,1643913161,50.9832,-118.2023,17.62,93,100,0.00
557,sanchursk,RU,1643913161,56.9413,47.2499,4.37,100,100,3.13


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.
* Drop any rows will null values.

In [3]:
#Configure gmaps
gmaps.configure(api_key = g_key)

In [4]:
#Cleaning data by dropping nan values
city_data_df = city_data_df.dropna()
city_data_df


,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,porto novo,BJ,1643912886,6.4965,2.6036,84.02,64,75,8.39
1,kapaa,US,1643912886,22.0752,-159.3190,71.92,77,20,11.50
2,itarema,BR,1643912887,-2.9248,-39.9167,85.23,65,78,16.22
3,pyapon,MM,1643912888,16.2833,95.6833,68.25,65,14,5.97
4,rikitea,PF,1643912683,-23.1203,-134.9692,77.36,77,100,14.41
...,...,...,...,...,...,...,...,...,...
554,aden,YE,1643913160,12.7794,45.0367,78.13,60,0,9.44
555,rutland,US,1643913008,43.6167,-73.0162,42.24,94,100,1.01
556,revelstoke,CA,1643913161,50.9832,-118.2023,17.62,93,100,0.00
557,sanchursk,RU,1643913161,56.9413,47.2499,4.37,100,100,3.13


In [5]:
#Use the Lat and Lng as locations
locations = city_data_df[["Lat","Lng"]].astype(float)
humidity = city_data_df["Humidity"]

In [6]:
# Create a humidity heatmap layer
fig = gmaps.figure(center = [0,0] ,zoom_level = 2)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.


In [7]:
#Narrow down the cities to fit weather conditions
perfect_weather_city_df = city_data_df.loc[(city_data_df["Max Temp"] <= 80)
                                          &(city_data_df["Max Temp"] >= 70)
                                          &(city_data_df["Wind Speed"] <= 10)
                                          &(city_data_df["Cloudiness"] == 0),:].dropna()
perfect_weather_city_df

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
76,mar del plata,AR,1643912682,-38.0023,-57.5575,73.87,83,0,4.61
105,ahuimanu,US,1643912934,21.4447,-157.8378,74.46,72,0,1.01
106,sao filipe,CV,1643912910,14.8961,-24.4956,75.20,68,0,7.65
258,makakilo city,US,1643913011,21.3469,-158.0858,73.29,69,0,4.61
292,pisco,PE,1643913028,-13.7000,-76.2167,74.01,78,0,8.05
325,douentza,ML,1643913045,14.9951,-2.9517,75.92,11,0,7.52
452,araouane,ML,1643913108,18.9048,-3.5265,71.91,9,0,9.48
467,kalmunai,LK,1643913115,7.4167,81.8167,74.93,78,0,2.15
554,aden,YE,1643913160,12.7794,45.0367,78.13,60,0,9.44


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df = perfect_weather_city_df.copy()
hotel_df["Hotel Name"] = " "
hotel_df

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Hotel Name
76,mar del plata,AR,1643912682,-38.0023,-57.5575,73.87,83,0,4.61,
105,ahuimanu,US,1643912934,21.4447,-157.8378,74.46,72,0,1.01,
106,sao filipe,CV,1643912910,14.8961,-24.4956,75.20,68,0,7.65,
258,makakilo city,US,1643913011,21.3469,-158.0858,73.29,69,0,4.61,
292,pisco,PE,1643913028,-13.7000,-76.2167,74.01,78,0,8.05,
325,douentza,ML,1643913045,14.9951,-2.9517,75.92,11,0,7.52,
452,araouane,ML,1643913108,18.9048,-3.5265,71.91,9,0,9.48,
467,kalmunai,LK,1643913115,7.4167,81.8167,74.93,78,0,2.15,
554,aden,YE,1643913160,12.7794,45.0367,78.13,60,0,9.44,


In [9]:
#Set parameters to search for hotels with 5000 meters.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 5000,
    "type": "hotel",
    "keyword": "lodging",
    "key": g_key
         }

In [10]:
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
     # add location to params dict
    params["location"] = f"{lat},{lng}"
    
    # assemble url and make API request
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
   

Retrieving Results for Index 76: mar del plata.
Closest hotel is Casa Daupi.
------------
Retrieving Results for Index 105: ahuimanu.
Closest hotel is Paradise Bay Resort.
------------
Retrieving Results for Index 106: sao filipe.
Closest hotel is Hotel Santos Pina.
------------
Retrieving Results for Index 258: makakilo city.
Closest hotel is Barber's Point and ITT and Beach Cottages.
------------
Retrieving Results for Index 292: pisco.
Closest hotel is Posada Gino.
------------
Retrieving Results for Index 325: douentza.
Closest hotel is Ogomono Doumbo.
------------
Retrieving Results for Index 452: araouane.
Missing field/result... skipping.
------------
Retrieving Results for Index 467: kalmunai.
Closest hotel is Villa LUXEZ.
------------
Retrieving Results for Index 554: aden.
Closest hotel is Coral Hotel.
------------


In [11]:
 hotel_df

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Hotel Name
76,mar del plata,AR,1643912682,-38.0023,-57.5575,73.87,83,0,4.61,Casa Daupi
105,ahuimanu,US,1643912934,21.4447,-157.8378,74.46,72,0,1.01,Paradise Bay Resort
106,sao filipe,CV,1643912910,14.8961,-24.4956,75.20,68,0,7.65,Hotel Santos Pina
258,makakilo city,US,1643913011,21.3469,-158.0858,73.29,69,0,4.61,Barber's Point and ITT and Beach Cottages
292,pisco,PE,1643913028,-13.7000,-76.2167,74.01,78,0,8.05,Posada Gino
325,douentza,ML,1643913045,14.9951,-2.9517,75.92,11,0,7.52,Ogomono Doumbo
452,araouane,ML,1643913108,18.9048,-3.5265,71.91,9,0,9.48,
467,kalmunai,LK,1643913115,7.4167,81.8167,74.93,78,0,2.15,Villa LUXEZ
554,aden,YE,1643913160,12.7794,45.0367,78.13,60,0,9.44,Coral Hotel


In [12]:
# NOTE: Do not change any of the code in this cell

    
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# Add the layer to the map
fig.add_layer(marker_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))